# Scrape Dashboard Data for COVID Cases 
## in Pinellas County Schools

In this notebook, we develop tools to scrape and analyze the data contained in Pinellas County School's COVID database. The tools include county wide totals, school by school analyses, and data visualization.

## Navigate to the url and click the submit button
This web page provides some searchability of the COVID results during the 2021-2022 school year in Pinellas County Schools. We do not need the searchability functionality of the webpage; we need the data contained in the database. To get the whole database, only only needs to click on the `Submit` button without any filters applied. Then the web page dynamically displays a table. 

The cell below loads packages, sets the correct url, and clicks the `Submit` button. After that, the notebook picks out the table and creates a dataframe. 



In [1]:
#Load packages
import requests
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait as WDW
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import lxml

#Set URL
URL = 'https://www.pcsb.org/covid19cases'
URL_2020_2021 = 'https://www.pcsb.org/Page/34025'




## Scraping the table into a df

Here we create a series of functions that can be used to scrape the data and advance through the pages of tables that are present. 

In [43]:
#Define constants and functions to get table and to go to the next page

ID = 'ui-paging-container'
DEBUG = False

def debug(*args):
    if DEBUG == True:
        print(args)

def get_table(driver):
    #Access table on each page
    soup = BeautifulSoup(driver.page_source, 'lxml')
    table = soup.find_all('table')

    #read the table
    new_df = pd.read_html(str(table))
    
    return new_df[0]

def get_page_indices(driver, ID):
    paging_buttons = driver.find_element(By.ID, ID).text
    page_text_indices = [page for page in paging_buttons.split('\n')]
    page_numbers = [int(page) for page in page_text_indices if (page != '...')]

    return paging_buttons, page_text_indices, page_numbers


def initiate_scraping(url, headless):
    #Set up selenium web interaction -
    options = webdriver.ChromeOptions()
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--incognito')
    if headless == True:
        options.add_argument('--headless')         #Operates webpage without viewing through Chrome
    driver = webdriver.Chrome("C:/webdrivers/chromedriver.exe")

    #Open webpage with webdriver, un-comment --headless argument above if you don't want to 
    #view the page. 
    driver.get(url)

    # Wait for pages to fully load for specified amount of time before throwing error.
    driver.implicitly_wait(10)

    #Now that the web page is open and operable, we need to click on the submit
    #button. Clicking on the search button allows us to get all of the data in a table. 
    driver.find_element_by_xpath('//*[@id="minibaseSubmit65979"]').click()

    return driver

def click_submit_main(driver):
    driver.find_element_by_xpath('//*[@id="minibaseSubmit65979"]').click()

def determine_total_pages(url, ID, headless = True):
    '''
    This function clicks the submit button, clicks on the ellipsis until the ellipsis is not the last
    in the list of page indices, and returns the total number of pages to be scraped of data tables.
    '''
    #Click submit without filters so that all data are displayed in table over multiple pages
    driver = initiate_scraping(url, headless)

    for x in range(10000):      #This covers up to 10000 pages of data tables on the website.
        _, page_text_indices, page_numbers = get_page_indices(driver, ID)
        if page_text_indices[-1] == '...':
            print('Iteration ', x, '. Clicking "..." to obtain pages after ', str(max(page_numbers)), ';\n searching for the last page.')
            next_page_xpath = '//*[@id="ui-paging-container"]/ul/li[' + str(max(page_numbers) + 1) + ']/a'
            print(next_page_xpath)
            button = driver.find_element_by_xpath(next_page_xpath)
            driver.execute_script("arguments[0].click();", button)
        else:
            total_pages = max(page_numbers)
            break
    
    new_search_xpath = '//*[@id="module-content-64809"]/div/div[2]/ul/li/div/div[1]/span/span/p[1]/a'
    new_search_button = driver.find_element_by_xpath(new_search_xpath)
    driver.execute_script("arguments[0].click();", new_search_button)

    return total_pages, page_text_indices, driver






In [44]:
#This cell uses some of the functions above, iterating through to the finish and concatenating the data.
#It takes approximately 3 seconds per table page to run, so expect long run times as the data grow larger.

DEBUG = True

#Determine the number of pages:
tot, indices, driver = determine_total_pages(URL, ID, headless=True)
print('Imitated filterless search to determine total pages: complete...\n')

#Initiate the page log:
data_df = pd.DataFrame([])
print('Empty DataFrame created. \n')

#Initiate filterless search
click_submit_main(driver)
print('Restarting filterless search to scrape data...')

#Set up dictionary container for data troubleshooting:
data_dict = {}
print('Empty dictionary ready for temporary data sets.\n')

#Scrape page 1 data
temp_df = get_table(driver)
data_df = pd.concat([data_df, temp_df])
key = 'Page ' + str(1)
data_dict |= {key:temp_df}
print('Data scraped from page 1 table...\n')
first_loop = True

#Loop through remaining pages, beginning with page 2
for page in range(2, tot+1):
    paging_buttons, page_text_indices, page_numbers = get_page_indices(driver, ID)
    if page <= max(page_numbers):
        if first_loop is True: 
            debug('Working in the if loop, first group of pages.')
            #This is the condition for the first time iterating through the table pages
            next_page_xpath = '//*[@id="ui-paging-container"]/ul/li[' + str(page) + ']/a'
            debug('Page 1 iteration ' + str(page) + ', xpath = ' + str(next_page_xpath))
            button = driver.find_element_by_xpath(next_page_xpath)
            driver.execute_script("arguments[0].click();", button)
            time.sleep(2)
        else:
            debug('Working in the if loop, subsequent groups of pages.')
            #This is the condition for the susequent times iterating through the table pages
            next_page_xpath = '//*[@id="ui-paging-container"]/ul/li[' + str(new_start) + ']/a'
            new_start = new_start + 1
            debug('Subsequent page iteration ' + str(page) + ', xpath = ' + str(next_page_xpath))
            button = driver.find_element_by_xpath(next_page_xpath)
            driver.execute_script("arguments[0].click();", button)
            time.sleep(2)
        #Scrape data and concatenate
        temp_df = get_table(driver)
        debug('Data scraped from page ' + str(page) + r' table ...\n')
        #debug(temp_df.head())
        data_df = pd.concat([data_df, temp_df])
        key = 'Page ' + str(page)
        data_dict |= {key:temp_df}

    elif page > max(page_numbers):
        debug('Working from the elif loop (clicking the ellipsis).')
        #In this case, we click the right ellipsis.
        next_page_xpath = '//*[@id="ui-paging-container"]/ul/li[' + str(page) + ']/a'
        debug('Page 1 iteration ' + str(page) + ', xpath = ' + str(next_page_xpath))
        button = driver.find_element_by_xpath(next_page_xpath)
        driver.execute_script("arguments[0].click();", button)
        time.sleep(2)
        #Scrape data and concatenate
        temp_df = get_table(driver)
        debug('Data scraped from page ' + str(page) + r' table ...\n, represented by "..."...')
        #debug(temp_df.head())
        data_df = pd.concat([data_df, temp_df])
        key = 'Page ' + str(page)
        data_dict |= {key:temp_df}
        #Set first_loop = False
        first_loop = False
        debug(page)
        debug(first_loop)
        new_start = 4
        debug('New counter for constructing x_paths: ', new_start)
 

Iteration  0 . Clicking "..." to obtain pages after  10 ;
 searching for the last page.
//*[@id="ui-paging-container"]/ul/li[11]/a
Iteration  1 . Clicking "..." to obtain pages after  10 ;
 searching for the last page.
//*[@id="ui-paging-container"]/ul/li[11]/a
Imitated filterless search to determine total pages: complete...

Empty DataFrame created. 

Restarting filterless search to scrape data...
Empty dictionary ready for temporary data sets.

Data scraped from page 1 table...

('Working in the if loop, first group of pages.',)
('Page 1 iteration 2, xpath = //*[@id="ui-paging-container"]/ul/li[2]/a',)
('Data scraped from page 2 table ...\\n',)
('Working in the if loop, first group of pages.',)
('Page 1 iteration 3, xpath = //*[@id="ui-paging-container"]/ul/li[3]/a',)
('Data scraped from page 3 table ...\\n',)
('Working in the if loop, first group of pages.',)
('Page 1 iteration 4, xpath = //*[@id="ui-paging-container"]/ul/li[4]/a',)
('Data scraped from page 4 table ...\\n',)
('Worki

In [45]:
print(data_df.shape)
print(data_dict.keys())



(296, 7)
dict_keys(['Page 1', 'Page 2', 'Page 3', 'Page 4', 'Page 5', 'Page 6', 'Page 7', 'Page 8', 'Page 9', 'Page 10', 'Page 11', 'Page 12', 'Page 13'])


In [41]:
pd.DataFrame.to_csv(data_df, 'data_dump_20210816')

In [42]:
#Slice the dataframe for the data
data_df['Total cases'] = sum(data_df['Number of positive employees'], data_df['Number of positive students'])
print(data_df.columns)
print('Total cases in district = ' + str(sum(data_df['Total cases'])))
print(data_df.groupby('Locations affected').sum().sort_values(by='Total cases', ascending=False))

Index([                             0,                              1,
                                    2,                         'Date',
                 'Locations affected', 'Number of positive employees',
        'Number of positive students',                  'Total cases'],
      dtype='object')
Total cases in district = nan
                                           Number of positive employees  \
Locations affected                                                        
Administration Building                                            17.0   
Palm Harbor University High School                                  1.0   
Ponce de Leon Elementary School                                     0.0   
Plumb Elementary School                                             1.0   
Pinellas Technical College St. Petersburg                           4.0   
...                                                                 ...   
Garrison-Jones Elementary School                               